# Setup

https://medium.com/swlh/revisiting-quicksort-with-julia-and-cuda-2a997447939b

In [3]:
# Pkg.add("Plots")
# Pkg.add("DataFrames")
# Pkg.add("StatsPlots")
# Pkg.add("CSV")

In [52]:
using Pkg
using Random
using SortingAlgorithms
using Plots
using DataFrames
using StatsPlots
using CSV
using CUDA
# using SortingSortingOut
using BenchmarkTools
using SortingAlgorithms
using CUDA

In [53]:
Pkg.activate("..")

  Activating environment at `~/Workspace/Repo/Code/SpinGlassExhaustive.jl/Project.toml`


In [54]:
# Pkg.add("SortingSortingOut")
Pkg.add("BenchmarkTools")
Pkg.add("CUDA")
Pkg.add("AbstractPlotting")
Pkg.add("Makie")

    Updating registry at `~/.julia/registries/General`
   Resolving package versions...
  No Changes to `~/Workspace/Repo/Code/SpinGlassExhaustive.jl/Project.toml`
  No Changes to `~/Workspace/Repo/Code/SpinGlassExhaustive.jl/Manifest.toml`
Precompiling project...
  ? NearestNeighbors
  ? PaddedViews
  ? DataStructures
  ? DiffResults
  ? StackViews
  ? PrettyTables
  ? Contour
  ? ChainRulesCore
  ? CSV
  ? TableOperations
  ? Interpolations
  ? LazyStack
  ? StructArrays
  ? QuadGK
  ? TiffImages
  ? SortingAlgorithms
  ? GPUCompiler
  ? NNlib
  ? SpecialFunctions
  ? ChainRules
  ? TensorCast
  ? ColorSchemes
  ? libass_jll
  ? GeometryBasics
  ? StatsBase
  ? CUDA
  ? StatsFuns
  ? DataFrames
  ? PlotUtils
  ? GridLayoutBase
  ? Packing
  ? ShaderAbstractions
  ? MeshIO
  ? MosaicViews
  ? MultivariateStats
  ? Distributions
  ? BinomialGPU
  ? Clustering
  ? PlotThemes
  ? RecipesPipeline
  ? ForwardDiff
  ? FFMPEG_jll
  ? KernelDensity
  ? ColorVectorSpace
  ? FFMPEG
  ? FreeType

  ? GridLayoutBase
  ? MosaicViews
  ? ShaderAbstractions
  ? BinomialGPU
  ? Clustering
  ? Distributions
  ? MultivariateStats
  ? PlotThemes
  ? RecipesPipeline
  ? ForwardDiff
  ? FFMPEG_jll
  ? KernelDensity
  ? ColorVectorSpace
  ? FFMPEG
  ? FreeTypeAbstraction
  ? QuantumInformation
  ? GR_jll
  ? Zygote
  ? Plots
  ? StatsPlots
  ? ImageCore
  ? Flux
  ? Netpbm
  ? CuQuantumInformation
  ? PNGFiles
  ? AbstractPlotting
  ? GLMakie
  ? Makie


# Benchmark

In [62]:
Random.seed!(1234)
n = 2^26

67108864

In [7]:
sort_arr = rand(Float32, n);
@benchmark sort!(arr; alg=QuickSort) setup=(arr=sort_arr)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     881.111 ms (0.00% GC)
  median time:      890.479 ms (0.00% GC)
  mean time:        897.694 ms (0.00% GC)
  maximum time:     920.161 ms (0.00% GC)
  --------------
  samples:          6
  evals/sample:     1

In [140]:
sort_arr = rand(Float32, n);
@benchmark sort!(arr; alg=MergeSort) setup=(arr=sort_arr)

BenchmarkTools.Trial: 
  memory estimate:  128.00 MiB
  allocs estimate:  3
  --------------
  minimum time:     1.299 s (0.00% GC)
  median time:      1.310 s (0.00% GC)
  mean time:        1.318 s (0.00% GC)
  maximum time:     1.352 s (0.00% GC)
  --------------
  samples:          4
  evals/sample:     1

In [141]:
sort_arr = rand(Float32, n);
@benchmark sort!(arr; alg=RadixSort) setup=(arr=sort_arr)

BenchmarkTools.Trial: 
  memory estimate:  256.09 MiB
  allocs estimate:  10
  --------------
  minimum time:     670.800 ms (0.00% GC)
  median time:      698.060 ms (0.00% GC)
  mean time:        696.101 ms (1.22% GC)
  maximum time:     721.420 ms (4.50% GC)
  --------------
  samples:          8
  evals/sample:     1

In [ ]:
# sort_arr = rand(Float32, n);
# @benchmark sort!(arr; alg=InsertionSort) setup=(arr=sort_arr)

In [63]:
shared = rand(Float32, n)
cushared = cu(shared)

@benchmark bitonicsort!(cushared)

BenchmarkTools.Trial: 
  memory estimate:  395.30 KiB
  allocs estimate:  10907
  --------------
  minimum time:     291.616 ms (0.00% GC)
  median time:      292.840 ms (0.00% GC)
  mean time:        292.788 ms (0.00% GC)
  maximum time:     293.252 ms (0.00% GC)
  --------------
  samples:          18
  evals/sample:     1

In [67]:
shared = rand(Float32, n)
cushared = cu(shared)

@benchmark bitonicsort2!(cushared)

BenchmarkTools.Trial: 
  memory estimate:  1.50 GiB
  allocs estimate:  2876
  --------------
  minimum time:     800.681 ms (3.53% GC)
  median time:      883.432 ms (8.68% GC)
  mean time:        895.753 ms (8.40% GC)
  maximum time:     976.530 ms (10.82% GC)
  --------------
  samples:          6
  evals/sample:     1

In [ ]:
shared = rand(Float32, n)
cushared = cu(shared)

@benchmark mergesort!(cushared)

In [60]:
arr = rand(Float32, n)
@benchmark partialsort!(arr, 1:10)

BenchmarkTools.Trial: 
  memory estimate:  48 bytes
  allocs estimate:  1
  --------------
  minimum time:     744.011 ms (0.00% GC)
  median time:      1.111 s (0.00% GC)
  mean time:        1.149 s (0.00% GC)
  maximum time:     1.514 s (0.00% GC)
  --------------
  samples:          5
  evals/sample:     1

In [59]:
shared = rand(Float32, 2^25)
cushared = CuArray(shared)

@benchmark CUDA.@sync(sort!(cushared))
# @benchmark sort!(cushared)

ERROR: Out of dynamic GPU memory (trying to allocate 8 bytes)
ERROR: a exception was thrown during kernel execution.
       Run Julia on debug level 2 for device stack traces.


LoadError: KernelException: exception thrown during kernel execution on device NVIDIA GeForce RTX 2080 Ti

In [13]:
shared = rand(Float32, 2^26)
cushared = CuArray(shared)

@benchmark CUDA.@sync(sort!(cushared))

ERROR: a CUDA error was thrown during kernel execution: invalid configuration argument (code 9, cudaErrorInvalidConfiguration)
ERROR: a exception was thrown during kernel execution.
       Run Julia on debug level 2 for device stack traces.


LoadError: KernelException: exception thrown during kernel execution on device NVIDIA GeForce RTX 2080 Ti

In [9]:
nn[1], nn[12], 2^26

(16, 67108864, 67108864)

In [31]:
shared = rand(Float32, n)
cushared = cu(shared)

@benchmark CUDA.@sync(partialsort!(cushared, 10))

a = rand(Float32, n)
c = CuArray(a)
partialsort!(c, 1:1000)

1000-element CuArray{Float32, 1}:
 0.0
 0.0
 0.0
 0.0
 0.0
 1.1920929f-7
 1.1920929f-7
 1.1920929f-7
 1.1920929f-7
 1.1920929f-7
 1.1920929f-7
 1.1920929f-7
 1.1920929f-7
 ⋮
 1.5377998f-5
 1.5377998f-5
 1.5377998f-5
 1.5497208f-5
 1.5497208f-5
 1.5497208f-5
 1.5497208f-5
 1.5497208f-5
 1.5616417f-5
 1.5616417f-5
 1.5616417f-5
 1.5616417f-5

In [22]:
x = rand(10)

partialsort!(x, 1:5)

5-element view(::Vector{Float64}, 1:5) with eltype Float64:
 0.06369545359322615
 0.0880627134662304
 0.140652278943995
 0.41678696591272235
 0.7987949387784006

In [23]:
x

10-element Vector{Float64}:
 0.06369545359322615
 0.0880627134662304
 0.140652278943995
 0.41678696591272235
 0.7987949387784006
 0.8095898830451091
 0.8696039258116677
 0.9079806880959223
 0.9202432584410771
 0.9995070347360671

In [ ]:
# c = CuArray(rand(Float32, (n, 1)))

# @benchmark sort!(c, rev=true, dims=2)

In [13]:
sort_arr = rand(Float32, n);
@benchmark CUDA.@sync(sort!(sort_arr; alg=QuickSort))

BenchmarkTools.Trial: 
  memory estimate:  160 bytes
  allocs estimate:  9
  --------------
  minimum time:     26.762 s (0.00% GC)
  median time:      26.762 s (0.00% GC)
  mean time:        26.762 s (0.00% GC)
  maximum time:     26.762 s (0.00% GC)
  --------------
  samples:          1
  evals/sample:     1

In [7]:
function quicksort_kernel(A, lo, hi)
    # blockDim().x is the size of the thread block
    # threadIdx().x returns the id of the thread that calls it
    swap = @cuDynamicSharedMem(T, blockDim().x)
    stride = L <= blockDim().x ? Int32(1) : Int32(L ÷ blockDim().x)
    brick_sort(A, lo, hi, swap, stride)
    if hi - lo <= blockDim().x
        return
    end
    pivot = A[lo + stride * blockDim().x ÷ 2]
    # TODO: partition
    # TODO: launch left/right quicksort_kernel
    return
end
#launch
# my_list = CuArray(rand(Float32, 1000))

shared = rand(Float32, 1000)
my_list = cu(shared)
@cuda threads=512 quicksort_kernel(my_list, 0, length(my_list))

LoadError: InvalidIRError: compiling kernel quicksort_kernel(CuDeviceVector{Float32, 1}, Int64, Int64) resulted in invalid LLVM IR
Reason: unsupported dynamic function invocation (call to *)
Stacktrace:
 [1] [0m[1mquicksort_kernel[22m
[90m   @ [39m[90;4mIn[7]:10[0m
Reason: unsupported dynamic function invocation (call to div)
Stacktrace:
 [1] [0m[1mquicksort_kernel[22m
[90m   @ [39m[90;4mIn[7]:10[0m
Reason: unsupported dynamic function invocation (call to +)
Stacktrace:
 [1] [0m[1mquicksort_kernel[22m
[90m   @ [39m[90;4mIn[7]:10[0m
Reason: unsupported dynamic function invocation (call to getindex)
Stacktrace:
 [1] [0m[1mquicksort_kernel[22m
[90m   @ [39m[90;4mIn[7]:10[0m
Reason: unsupported use of an undefined name (use of 'T')
Stacktrace:
 [1] [0m[1mmacro expansion[22m
[90m   @ [39m[90m~/.julia/packages/CUDA/M4jkK/src/device/intrinsics/[39m[90;4mmemory_shared.jl:49[0m
 [2] [0m[1mquicksort_kernel[22m
[90m   @ [39m[90;4mIn[7]:4[0m
Reason: unsu

# Benchmark 2

In [6]:
nn = [2^(2*n) for n in 2:13];

In [58]:
qs_times = []

for n in nn
    sort_arr = rand(Float32, n);
    a = @benchmark sort!(arr; alg=QuickSort) setup=(arr=sort_arr)
    ma = mean(a.times)
    
    print(a)
    print(ma)
    print(" ")
    append!(qs_times, ma)
end

Trial(41.309 ns)42.36917598385469 Trial(173.976 ns)183.4832059474412 Trial(848.188 ns)874.1158652173914 Trial(3.890 μs)4014.7115875 Trial(17.135 μs)17869.4838 Trial(75.192 μs)78665.2084 Trial(330.708 μs)347456.0031 Trial(1.474 ms)1.525280749617854e6 Trial(6.498 ms)7.090094186079546e6 Trial(32.044 ms)3.5450530326241136e7 Trial(165.506 ms)1.744369988275862e8 Trial(942.452 ms)9.878016905e8 

In [59]:
ms_times = []

for n in nn
    sort_arr = rand(Float32, n);
    a = @benchmark sort!(arr; alg=MergeSort) setup=(arr=sort_arr)
    ma = mean(a.times)
    
    print(a)
    print(ma)
    print(" ")
    append!(ms_times, ma)
end

Trial(76.285 ns)107.64795787847582 Trial(282.216 ns)373.46612328767117 Trial(1.296 μs)1521.13377 Trial(6.626 μs)8203.554059999999 Trial(29.345 μs)34323.2657 Trial(136.266 μs)158184.0827 Trial(689.697 μs)732902.7178882171 Trial(3.135 ms)3.3291091416165666e6 Trial(14.255 ms)1.5283055867692309e7 Trial(67.236 ms)7.396225226470588e7 Trial(293.541 ms)3.178968831875e8 Trial(1.307 s)1.3259118145e9 

In [57]:
rs_times = []

for n in nn
    sort_arr = rand(Float32, n);
    a = @benchmark sort!(arr; alg=RadixSort) setup=(arr=sort_arr)
    ma = mean(a.times)
    
    print(a)
    print(ma)
    print(" ")
    append!(rs_times, ma)
end

Trial(8.443 μs)13759.9098 Trial(8.436 μs)12916.706 Trial(9.523 μs)13074.1998 Trial(15.157 μs)19745.8015 Trial(34.224 μs)39610.9536 Trial(122.291 μs)131129.4733 Trial(495.881 μs)524993.7901351922 Trial(1.958 ms)2.0443216339762001e6 Trial(8.085 ms)8.903976586452764e6 Trial(36.537 ms)3.809958456060606e7 Trial(161.457 ms)1.672049787e8 Trial(677.494 ms)7.023265915e8 

In [21]:
nn2 = [2^(2*n) for n in 2:12];
js_times = []

for n in nn2

    shared = rand(Float32, n)
    cushared = CuArray(shared)

    a = @benchmark CUDA.@sync(sort!(cushared))
    ma = mean(a.times)
    
    print(a)
    print(ma)
    print(" ")
    append!(js_times, ma)
end


Trial(28.023 μs)62488.0193 Trial(36.057 μs)69632.2472 Trial(78.951 μs)116692.7342 Trial(3.751 ms)4.460044367713004e6 Trial(6.180 ms)6.631771914893617e6 Trial(7.239 ms)8.443749780033842e6 Trial(10.415 ms)1.1807787203309692e7 Trial(17.439 ms)1.812927906884058e7 Trial(43.750 ms)4.6393472722222224e7 Trial(159.415 ms)1.6240744303225806e8 Trial(639.504 ms)6.49382007375e8 

In [40]:
# append!(js_times, NaN)
js_times[12]=0
js_times = convert(Array{Float64,1}, js_times)

12-element Vector{Float64}:
  62488.0193
  69632.2472
 116692.7342
      4.460044367713004e6
      6.631771914893617e6
      8.443749780033842e6
      1.1807787203309692e7
      1.812927906884058e7
      4.6393472722222224e7
      1.6240744303225806e8
      6.49382007375e8
      0.0

In [49]:
2^24, 2^26

(16777216, 67108864)

In [45]:
nn2 = [2^(2*n) for n in 2:12];
js_times2 = []

for n in nn2

    shared = rand(Float32, n)
    cushared = CuArray(shared)

    a = @benchmark CUDA.@sync(sort!(cushared))
    ma = mean(a.times)
    
    print(a)
    print(ma)
    print(" ")
    append!(js_times2, ma)
end

Trial(28.672 μs)58692.8328 Trial(38.970 μs)70758.9614 Trial(80.392 μs)111129.1335 Trial(6.788 ms)7.465375291916167e6 Trial(9.491 ms)1.0410745020876827e7 Trial(10.001 ms)1.210556809708738e7 ERROR: Out of dynamic GPU memory (trying to allocate 16 bytes)
ERROR: a exception was thrown during kernel execution.
       Run Julia on debug level 2 for device stack traces.
ERROR: Out of dynamic GPU memory (trying to allocate 16 bytes)
ERROR: a exception was thrown during kernel execution.
       Run Julia on debug level 2 for device stack traces.


LoadError: KernelException: exception thrown during kernel execution on device NVIDIA GeForce RTX 2080 Ti

In [135]:
df = DataFrame(n=nn, QuickSort = qs_times, MergeSort = ms_times, RadixSort = rs_times, BitonicSort=bs_times);

In [43]:
plotly()
@df df plot(:n, [:QuickSort :MergeSort :RadixSort :BitonicSort :GPUJuliaSort], title = "Czas sortowania", xlabel="rozmiar tablicy n", ylabel="czas w ns", lw = 3) # , label=["Quick Sort", "Merge Sort", "Radix Sort"]

<!DOCTYPE html>
 
 
 Plots.jl

In [165]:
CSV.write("sort_times.csv", df)

"sort_times.csv"

In [24]:
df = CSV.read("sort_times.csv", DataFrame)

,n,QuickSort,MergeSort,RadixSort,BitonicSort
,Int64,Float64,Float64,Float64,Float64
1,16,42.3692,107.648,13759.9,0.0
2,64,183.483,373.466,12916.7,0.0
3,256,874.116,1521.13,13074.2,1.45554e5
4,1024,4014.71,8203.55,19745.8,2.06584e5
5,4096,17869.5,34323.3,39611.0,2.85016e5
6,16384,78665.2,1.58184e5,1.31129e5,378903.0
7,65536,347456.0,7.32903e5,5.24994e5,4.8088e5
8,262144,1.52528e6,3.32911e6,2.04432e6,6.83008e5
9,1048576,7.09009e6,1.52831e7,8.90398e6,1.81625e6


In [41]:
df = DataFrame(n=nn, QuickSort = df[!,"QuickSort"], MergeSort = df[!,"MergeSort"], RadixSort = df[!,"RadixSort"], BitonicSort=df[!,"BitonicSort"], GPUJuliaSort=js_times);

In [42]:
df

,n,QuickSort,MergeSort,RadixSort,BitonicSort,GPUJuliaSort
,Int64,Float64,Float64,Float64,Float64,Float64
1,16,42.3692,107.648,13759.9,0.0,62488.0
2,64,183.483,373.466,12916.7,0.0,69632.2
3,256,874.116,1521.13,13074.2,1.45554e5,1.16693e5
4,1024,4014.71,8203.55,19745.8,2.06584e5,4.46004e6
5,4096,17869.5,34323.3,39611.0,2.85016e5,6.63177e6
6,16384,78665.2,1.58184e5,1.31129e5,378903.0,8.44375e6
7,65536,347456.0,7.32903e5,5.24994e5,4.8088e5,1.18078e7
8,262144,1.52528e6,3.32911e6,2.04432e6,6.83008e5,1.81293e7
9,1048576,7.09009e6,1.52831e7,8.90398e6,1.81625e6,4.63935e7


In [125]:
# bs_times = []

# for n in nn
#     shared = rand(Float32, n)
#     cushared = cu(shared)

#     a = @benchmark bitonicsort!(cushared)
#     ma = mean(a.times)
    
#     print(n)
#     print(" ")
#     print(ma)
#     print(" ")
#     append!(bs_times, ma)
# end

shared = rand(Float32, nn[12])
cushared = cu(shared)

a = @benchmark bitonicsort!(cushared)
ma = mean(a.times)

print(n)
print(" ")
print(ma)
print(" ")
append!(bs_times, ma)

67108864 2.939187552777778e8 

10-element Vector{Any}:
 145554.4752
 206584.4678
 285015.8622
 378903.0477
 480880.1595
 683007.69656883
      1.8162506839416057e6
      1.4074233828169014e7
      6.397147298734177e7
      2.939187552777778e8

In [131]:
pushfirst!(bs_times, 0)

12-element Vector{Any}:
      0
      0
 145554.4752
 206584.4678
 285015.8622
 378903.0477
 480880.1595
 683007.69656883
      1.8162506839416057e6
      1.4074233828169014e7
      6.397147298734177e7
      2.939187552777778e8

In [26]:
sort_arr = rand(Float32, nn[2]);
a = @benchmark sort!(arr; alg=QuickSort) setup=(arr=sort_arr)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     173.422 ns (0.00% GC)
  median time:      176.636 ns (0.00% GC)
  mean time:        184.797 ns (0.00% GC)
  maximum time:     544.871 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     728

In [43]:
mean(a.times)

184.79710934065935

# Bitonicsort GPU

https://en.wikipedia.org/wiki/Bitonic_sorter

In [57]:
# sort kernel
bisort!(shared, j, k) = begin
  tid = UInt(((blockIdx().x - 1) * blockDim().x + threadIdx().x)-1)
  ixj = tid⊻UInt(j)
  if ixj > tid
    if (tid & k) == 0
        if shared[tid+1] > shared[ixj+1]
          tmp = shared[ixj+1]
          shared[ixj+1] = shared[tid+1]
          shared[tid+1] = tmp
        end
    else
        if shared[tid+1] < shared[ixj+1]
          tmp = shared[ixj+1]
          shared[ixj+1] = shared[tid+1]
          shared[tid+1] = tmp
        end
    end
  end
  return
end

n_threat = 256

bitonicsort!(cushared) = begin
  CUDA.@sync begin
    k = UInt(2)
    NUM = length(cushared)
    nblocks = ceil(NUM/n_threat) |> Int
    while (k <= NUM)
      j = div(k, 2)
      while j >= 1
        @cuda threads = n_threat blocks = nblocks bisort!(cushared, j, k)
        j =  div(j,2)
      end
      k = UInt(k*2)
    end
  end
end

bitonicsort! (generic function with 1 method)

In [6]:
shared = rand(Float32, 2^26)

using SortingAlgorithms
shared = rand(Float32, 2^26)
@btime sort!($shared, alg=RadixSort) #735.216

shared = rand(Float32, 2^26)



using BenchmarkTools
res = Float64[]
using Flux
for i = 1:10
  cushared = cu(shared)
#   cpu(cushared) |> issorted |> println
  t = Base.@elapsed bitonicsort!(cushared)
#   cpu(cushared) |> issorted |> println
  push!(res, t)
end

res

  708.839 ms (10 allocations: 256.09 MiB)


10-element Vector{Float64}:
 9.993438632
 0.295888018
 0.295029303
 0.309847274
 0.295880691
 0.319390669
 0.313881531
 0.324470671
 0.3172449
 0.305680283

In [7]:
shared = rand(Float32, 2^26)
cushared = cu(shared)
bitonicsort!(cushared)

In [8]:
shared = rand(Float32, 2^26)
cushared = cu(shared)
bitonicsort!(cushared)
@benchmark bitonicsort!(cushared)

BenchmarkTools.Trial: 
  memory estimate:  98.88 KiB
  allocs estimate:  3168
  --------------
  minimum time:     293.048 ms (0.00% GC)
  median time:      293.547 ms (0.00% GC)
  mean time:        293.597 ms (0.00% GC)
  maximum time:     294.007 ms (0.00% GC)
  --------------
  samples:          18
  evals/sample:     1

# BitonicSort2

In [61]:
# Code based on: https://discourse.julialang.org/t/gpu-sort-wip-gpu-1000x-faster-than-cpu-i-must-be-doing-something-wrong/20310/4
# bitonic sorts a vector of any length.
using CUDA
function bisort!(shared, j, k) 
    tid = UInt(((blockIdx().x - 1) * blockDim().x + threadIdx().x)-1)
    ixj = tid⊻UInt(j)
    if ixj > tid
        if (tid & k) == 0
            if shared[tid+1] > shared[ixj+1]
                tmp = shared[ixj+1]
                shared[ixj+1] = shared[tid+1]
                shared[tid+1] = tmp
            end
        else
            if shared[tid+1] < shared[ixj+1]
                tmp = shared[ixj+1]
                shared[ixj+1] = shared[tid+1]
                shared[tid+1] = tmp
            end
        end
    end
    return
end

function bitonicsort_pow2!(cushared) 
    CUDA.@sync begin
        k = UInt(2)
        NUM = length(cushared)
        nblocks = ceil(NUM/256) |> Int
        while (k <= NUM)
            j = div(k, 2)
            while j >= 1
                @cuda threads = 256 blocks = nblocks bisort!(cushared, j, k)
                j =  div(j,2)
            end
            k = UInt(k*2)
        end
    end
end

function bitonicsort2!(cushared)
    N = length(cushared)
    K = convert(Int, ceil(log2(N)))
    paddedlen = 1<<K
    cushared2 = similar(cushared, paddedlen)
    fill!(cushared2, Inf)
    cushared2[1:N] .= @view cushared[1:N]
    bitonicsort_pow2!(cushared2)
    cushared .= @view cushared2[1:N]
    cushared
end

bitonicsort2! (generic function with 1 method)

# Merge Sort

In [65]:
# using CUDA
function gpu_bottomUpMerge!(source, dest, first, middle, last)
    i = first
    j = middle
    for k = first:last
        if i < middle && (j > last || source[i] < source[j])
            dest[k] = source[i]
            i += 1
        else
            dest[k] = source[j]
            j += 1
        end
    end
end

function gpu_mergesort!(source, dest, size, width, slices)
    idx = (blockIdx().x - 1) * blockDim().x + threadIdx().x 
    first = width * (idx - 1) * slices + 1
    for slice = 1:slices
        if first > size
            break
        end
        middle = min(first + (width >> 1), size)
        last = min(first + width - 1, size)
        gpu_bottomUpMerge!(source, dest, first, middle, last)
        first += width
    end
end

function mergesort!(data; verbose=false)
    size = length(data)
    data_swap = similar(data)
    A = data
    B = data_swap
    nThreads = 512
    width = 2
    while width < (size << 1)
        slices = size ÷ (nThreads * width) + 1
        if verbose
            println("mergesort - width: $width, slices: $slices, nThreads: $nThreads")
        end
        @cuda threads=nThreads gpu_mergesort!(A, B, size, width, slices)
        A, B = B, A
        width <<= 1
    end
    return A
end

mergesort! (generic function with 1 method)

# Quicksort GPU

In [44]:
const MAX_DEPTH = 16
const SELECTION_SORT = 32

function selection_sort(data, left, right)
    @inbounds for i in left:right
        min_val = data[i]
        min_idx = i

        # find the smallest value in the range [left, right]
        for j in i+1:right
            val_j = data[j]
            if val_j < min_val
                min_idx = j
                min_val = val_j
            end
        end

        # swap the values
        if i != min_idx
            data[min_idx] = data[i]
            data[i] = min_val
        end
    end
end

function quicksort(data, left, right, depth)
    # If we're too deep or there are few elements left, we use an insertion sort...
    if depth >= MAX_DEPTH || right-left <= SELECTION_SORT
        selection_sort(data, left, right)
        return
    end

    lptr = left
    rptr = right
    pivot = @inbounds data[(left+right)÷2]

    # do the partitioning
    @inbounds while lptr <= rptr
        # find the next left- and right-hand values to swap
        lval = data[lptr]
        rval = data[rptr]

        # move the left pointer as long as the pointed element is smaller than the pivot
        while lval < pivot && lptr < right
            lptr += 1
            lval = data[lptr]
        end

        # move the right pointer as long as the pointed element is larger than the pivot
        while rval > pivot && rptr > left
            rptr -= 1
            rval = data[rptr]
        end

        # if the swap points are valid, do the swap!
        if lptr <= rptr
            data[lptr] = rval
            data[rptr] = lval
            lptr += 1
            rptr -= 1
        end
    end

    # launch a new block to sort the left part
    if left < rptr
        s = CuDeviceStream()
        @cuda dynamic=true stream=s quicksort(data, left, rptr, depth+1)
        CUDAnative.unsafe_destroy!(s)
    end

    # launch a new block to sort the right part
    if lptr < right
        s = CuDeviceStream()
        @cuda dynamic=true stream=s quicksort(data, lptr, right, depth+1)
        CUDAnative.unsafe_destroy!(s)
    end

    return
end

function Base.sort!(a::CuArray)
    limit!(CUDAdrv.LIMIT_DEV_RUNTIME_SYNC_DEPTH, MAX_DEPTH)
    left = 1
    right = length(a)
    @cuda quicksort(a, left, right, 0)
    return a
end


using Test
using BenchmarkTools

function main(N=1000)
    a = rand(N)

    bm = @benchmark sort!(a) setup=(a = copy($a))
    println(1)
    println(mean(bm.times))
    
    b = CuArray(a)
    @test sort!(copy(a)) == Array(sort!(copy(b)))

#     CUDA.@profile for i in 1:10
#         sort!(copy(b))
#     end

    bm = @benchmark CUDA.@sync(sort!(b)) setup=(b = copy($b))
    println(2)
    println(mean(bm.times))
    
    @btime begin
        x = Array(b)
        sort!(b)
        copyto!(x, b)
    end setup=(b = copy($b))

    return
end

function quicksort_gpu!(a)
    b = CuArray(a)

    a = @benchmark CUDA.@sync(sort!(b)) setup=(b = copy($b))
    println(mean(a))
    
    return
end

isinteractive() || main()

true

In [ ]:
[62828.8007 80786.3343 213027.2962 6.086803043209877e6 9.825459642857144e6 1.2431816874686716e7 1.6527301823333334e7]

In [27]:
for n in nn
    main(n)
    println("###")
end

1
35.63391468812877
2
62828.8007
  32.865 μs (41 allocations: 1.66 KiB)
###
1
175.1316426923077
2
80786.3343
  42.124 μs (41 allocations: 2.06 KiB)
###
1
909.4657957142856
2
213027.2962
  139.931 μs (41 allocations: 3.58 KiB)
###
1
5885.1503625
2
6.086803043209877e6
  5.663 ms (45 allocations: 9.64 KiB)
###
1
137987.5418
2
9.825459642857144e6
  9.238 ms (46 allocations: 33.59 KiB)
###
1
641385.320446393
2
1.2431816874686716e7
  11.126 ms (46 allocations: 129.59 KiB)
###
1
3.043778391441158e6
2
1.6527301823333334e7
  16.545 ms (46 allocations: 513.59 KiB)
###
1
1.4156345249266863e7
ERROR: Out of dynamic GPU memory (trying to allocate 16 bytes)
ERROR: a exception was thrown during kernel execution.
       Run Julia on debug level 2 for device stack traces.


LoadError: CUDA error: an illegal memory access was encountered (code 700, ERROR_ILLEGAL_ADDRESS)

In [29]:
main(nn[8])

1
1.350538144537815e7


LoadError: CUDA error: an illegal memory access was encountered (code 700, ERROR_ILLEGAL_ADDRESS)

In [21]:
shared = rand(Float32, 1000)

quicksort_gpu!(CuArray(shared))

LoadError: CUDA error: out of memory (code 2, ERROR_OUT_OF_MEMORY)

In [19]:
shared = rand(Float32, 2^26)
cushared = cu(shared)

@benchmark bitonicsort!(cushared)

BenchmarkTools.Trial: 
  memory estimate:  411.75 KiB
  allocs estimate:  11258
  --------------
  minimum time:     290.141 ms (0.00% GC)
  median time:      293.546 ms (0.00% GC)
  mean time:        293.503 ms (0.00% GC)
  maximum time:     294.333 ms (0.00% GC)
  --------------
  samples:          18
  evals/sample:     1

In [32]:
# shared = rand(Float32, 2^26)
# cushared = cu(shared)

# @benchmark quicksort!(cushared)

x = CUDA.rand(10);

partialsort(x, 1:3)

┌ Warning: Performing scalar operations on GPU arrays: This is very slow, consider disallowing these operations with `allowscalar(false)`
└ @ GPUArrays /home/dk/.julia/packages/GPUArrays/WV76E/src/host/indexing.jl:43


3-element CuArray{Float32, 1}:
 0.0012321864
 0.014328189
 0.25416237

In [20]:
2^26

67108864

In [28]:
function main2(T, N)
    b = @benchmarkable (CUDA.@sync(sort!(c))) setup=(c = CuArray(rand($T, $N))) samples=7 seconds=30
    run(b)
end

main2(Float64, 10000000)

BenchmarkTools.Trial: 
  memory estimate:  1.66 KiB
  allocs estimate:  50
  --------------
  minimum time:     591.056 ms (0.00% GC)
  median time:      605.127 ms (0.00% GC)
  mean time:        612.299 ms (0.00% GC)
  maximum time:     650.500 ms (0.00% GC)
  --------------
  samples:          7
  evals/sample:     1

In [26]:
function quicksort_gpu!(a)
    b = CuArray(a)
    limit!(CUDAdrv.LIMIT_DEV_RUNTIME_SYNC_DEPTH, MAX_DEPTH)
    left = 1
    right = length(b)
    @cuda quicksort(b, left, right, 0)
    return 
end

quicksort_gpu! (generic function with 2 methods)

# Thrust

https://thrust.github.io/

```
// generate 32M random numbers serially
thrust::host_vector<int> h_vec(32 << 20);
  
std::generate(h_vec.begin(), h_vec.end(), rand);

// transfer data to the device
thrust::device_vector<int> d_vec = h_vec;

// sort data on the device (846M keys per second on GeForce GTX 480)   thrust::sort(d_vec.begin(), d_vec.end());

// transfer data back to host
thrust::copy(d_vec.begin(), d_vec.end(), h_vec.begin());
```